In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snp

In [ ]:
train_data = pd.read_csv('../input/covid19-global-forecasting-week-2/train.csv')
test_data = pd.read_csv('../input/covid19-global-forecasting-week-2/test.csv')

In [ ]:
train_data.head(5)

In [ ]:
    train_data.describe()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.fillna('Other', inplace=True)
test_data.fillna('Other', inplace=True)

In [ ]:
plt.figure(figsize=(10,25))
snp.barplot(x='ConfirmedCases', y='Province_State', data=train_data)

In [ ]:
train_data.head(2)

In [ ]:
test_data.head(2)

In [ ]:
import datetime

In [ ]:
train_data['month'] = pd.DatetimeIndex(train_data['Date']).month
train_data['day'] = pd.DatetimeIndex(train_data['Date']).day
test_data['month'] = pd.DatetimeIndex(test_data['Date']).month
test_data['day'] = pd.DatetimeIndex(test_data['Date']).day
train_data.drop('Date', axis=1, inplace=True)
test_data.drop('Date', axis=1, inplace=True)

In [ ]:
a = pd.get_dummies(train_data['Country_Region'], drop_first=True)
b = pd.get_dummies(train_data['Province_State'], drop_first=True)
train_data.drop(['Country_Region','Province_State'], axis=1, inplace=True)
train_data = pd.concat([train_data, a, b], axis=1)
c = pd.get_dummies(test_data['Country_Region'], drop_first=True)
d = pd.get_dummies(test_data['Province_State'], drop_first=True)
test_data.drop(['Country_Region','Province_State'], axis=1, inplace=True)
test_data = pd.concat([test_data, c, d], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = train_data.drop(['ConfirmedCases', 'Fatalities'], axis=1)
y1 = train_data['ConfirmedCases']
y2 = train_data['Fatalities']

In [ ]:
df = pd.DataFrame(test_data['ForecastId'])

In [ ]:
scalar = StandardScaler()

In [ ]:
X = scalar.fit_transform(X)
test_data = scalar.transform(test_data)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
model = RandomForestRegressor(n_estimators=100, bootstrap=True, max_features='auto', min_samples_split=2, random_state=40)

In [ ]:
model.fit(X, y1)

In [ ]:
prediction = model.predict(test_data)

In [ ]:
df['ConfirmedCases'] = pd.DataFrame(prediction)

In [ ]:
new_model = RandomForestRegressor(n_estimators=100, bootstrap=True, max_features='auto', min_samples_split=2, random_state=40)

In [ ]:
new_model.fit(X,y2)

In [ ]:
df['Fatalities'] = new_model.predict(test_data)

In [ ]:
df['ForecastId'] = df['ForecastId'].astype(int)
df['ConfirmedCases'] = df['ConfirmedCases'].astype(int)
df['Fatalities'] = df['Fatalities'].astype(int)

In [ ]:
df.to_csv('submission.csv', index=False)